In [1]:
import pandas as pd

In [3]:
# Convert excel to csv and txt

census_file = pd.read_excel("census_local_unit_age.xlsx")
census_file.to_csv('census_local_unit_age.txt', index=False, encoding='utf-8')

In [4]:
data = pd.read_csv('census_local_unit_age.txt')
print(data.head(200))

    Table 02: Population by five year age groups, NPHC 2021 Unnamed: 1   
0                                                  NaN             NaN  \
1                                                Area              NaN   
2                                                  NaN             NaN   
3                                                KOSHI             NaN   
4                                                  NaN       Taplejung   
..                                                 ...             ...   
195                                                NaN             NaN   
196                                                NaN             NaN   
197                                                NaN             NaN   
198                                                NaN             NaN   
199                                                NaN             NaN   

    Unnamed: 2 Unnamed: 3             Unnamed: 4 Unnamed: 5 Unnamed: 6   
0          NaN        NaN            

In [32]:
import re
import csv

with open('census_local_unit_age.txt') as file, open('output_filtered.csv', 'w') as out_file:
    out_header_row = ['province','district', 'municipality', 'ward', 'gender', 'all_ages', '00-04','05-09','10-14','15-19','20-24','25-29',
                      '30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85-89','90-94','95+']
    writer = csv.writer(out_file)
    writer.writerow(out_header_row)
    
    
    province = "Koshi"        
    district = ""
    municipality = ""
    ward = ""

    total = []
    male = []
    female = []
    for index, line in enumerate(file):
        # print(index)
        
        if index <= 4:
            continue
        if line[1] != ",":
            district = line.replace(",", "").replace("\n", "")
        if line[2] != "," and line[1] == ",":
            previous_mun = municipality
            municipality = line.replace(",", "").replace("\n", "")
            
        if line[3] != "," and line[2] == ",":   
            temp_ward = line.replace(",","").replace("\n", "")
            out_mun = ""
            if len(total)>0:                
                if "All" in temp_ward:
                    out_mun = previous_mun
                elif "All" not in temp_ward:
                    out_mun = municipality
                    # write to the row                
                total_row = [province, district, out_mun, ward, 'all'] + total
                # print(total_row)
                male_row = [province, district, out_mun, ward, 'male' ] + male
                female_row = [province, district, out_mun, ward, 'female'] + female
                writer.writerow(total_row)
                writer.writerow(male_row)
                writer.writerow(female_row)
                # print(female_row)
                total.clear()
                male.clear()
                female.clear()               
            ward = line.replace(",","").replace("\n", "")
            
        
        if line[4] != "," and line[3] == ",":
            line_temp = line[4:]
            line_data_list = line_temp.split(",")
            # print(line_data_list)
            total.append(line_data_list[1])
            male.append(line_data_list[2])
            female_temp_data = line_data_list[3].replace("\n", "")
            female.append(female_temp_data)
    